# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    
    source_text
    """
    # TODO: Implement Function
    
    eos = target_vocab_to_int['<EOS>'] 
    source_id_text = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_text.split("\n")]
    target_id_text = [[target_vocab_to_int[word] for word in sentence.split()] +[eos] for sentence in target_text.split("\n")]
    


    return source_id_text,target_id_text


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    
    inputs = tf.placeholder(dtype=tf.int32, shape=[None, None], name="input")
    targets = tf.placeholder(dtype=tf.int32, shape=[None, None], name="targets")
    learning_rate = tf.placeholder(dtype=tf.float32, name="learningrate")
    keep_prob = tf.placeholder(dtype=tf.float32, name="keep_prob")
    target_seq_length = tf.placeholder(dtype=tf.int32, shape=[None], name="target_sequence_length")
    max_target_seq_length = tf.reduce_max(target_seq_length)

    source_seq_length = tf.placeholder(dtype=tf.int32, shape=[None], name="source_sequence_length")
    # TODO: Implement Function
    return inputs, targets, learning_rate, keep_prob, target_seq_length, max_target_seq_length, source_seq_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    
    go = target_vocab_to_int['<GO>'] 
    target = tf.strided_slice(target_data,[0,0], [batch_size,-1],[1,1]) 
    target = tf.concat([tf.fill(dims=[batch_size,1],value=go),target],1) 
    
    
    return target

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    embed = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, embed_dim=encoding_embedding_size)

    

    # RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell,input_keep_prob=keep_prob,dtype=tf.int32)

        return enc_cell

    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    

    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, embed, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return enc_output, enc_state
    
    
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length=target_sequence_length,
                                                            time_major=False)
    
    
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       encoder_state,
                                                       output_layer)
    
    dynamic_decode_output, dynamic_decode_state = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                       impute_finished = True,
                                                       maximum_iterations = max_summary_length)
    
    
    
    return dynamic_decode_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    with tf.variable_scope("decode", reuse=True):
        start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')
        
    greedy_embed = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,start_tokens,end_of_sequence_id)
    basic_dec = tf.contrib.seq2seq.BasicDecoder(dec_cell,greedy_embed,encoder_state,output_layer)
    inference_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(basic_dec,impute_finished=True, maximum_iterations=max_target_sequence_length)
    

    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    

    # TODO: Implement Function

    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_inputs = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # Construct the decoder LSTM cell (just like you constructed the encoder cell above)
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell,input_keep_prob=keep_prob,dtype=tf.int32)

        return dec_cell
    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])

    output_layer = Dense(target_vocab_size, kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))



    with tf.variable_scope("decode"):

        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_inputs,
                                                            sequence_length=target_sequence_length,
                                                            time_major=False)


        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                           training_helper,
                                                           encoder_state,
                                                           output_layer) 

        training_decoder_output, _= tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                                       impute_finished=True,
                                                                       maximum_iterations=max_target_sequence_length)
    # inference decoder
    with tf.variable_scope("decode", reuse=True):
        start_tokens = tf.tile(tf.constant([target_vocab_to_int['<GO>']], dtype=tf.int32), [batch_size], name='start_tokens')

        # Helper for the inference process.
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                start_tokens,
                                                                target_vocab_to_int['<EOS>'])

        # Basic decoder
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        encoder_state,
                                                        output_layer)

        # Perform dynamic decoding using the decoder
        inference_decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            impute_finished=True,
                                                            maximum_iterations=max_target_sequence_length)



    return training_decoder_output, inference_decoder_output

    
    


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
        

#     emb_input_data = tf.contrib.layers.embed_sequence(input_data,source_vocab_size, enc_embedding_size)
    
#     enc_st = encoding_layer(emb_input_data, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, enc_embedding_size)
    
#     dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
#     emb_dec_input_data = tf.contrib.layers.embed_sequence(dec_input,source_vocab_size, enc_embedding_size)

#     dec_enc_input = decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)
    
#     train_logits, refer_logits = decoding_layer(
#         dec_embed_input, dec_embeddings, enc_state, target_vocab_size, 
#         sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)


    _, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, 
                       source_sequence_length, source_vocab_size, enc_embedding_size)


    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)

    return decoding_layer(dec_input, enc_state,
                       target_sequence_length, tf.reduce_max(target_sequence_length),
                       rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                       batch_size, keep_prob, dec_embedding_size)



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [28]:
# # Number of Epochs
# epochs = None
# # Batch Size
# batch_size = None
# # RNN Size
# rnn_size = None
# # Number of Layers
# num_layers = None
# # Embedding Size
# encoding_embedding_size = None
# decoding_embedding_size = None
# # Learning Rate
# learning_rate = None
# # Dropout Keep Probability
# keep_probability = None
# display_step = None

epochs = 2 

batch_size = 256 

rnn_size = 512 

num_layers = 2 
encoding_embedding_size = 256 
decoding_embedding_size = 256 
learning_rate = 0.001 
keep_probability = 0.5
display_step = 1

### Build the Graph
Build the graph using the neural network you implemented.

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [31]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/538 - Train Accuracy: 0.2684, Validation Accuracy: 0.3466, Loss: 4.9362
Epoch   0 Batch    2/538 - Train Accuracy: 0.2873, Validation Accuracy: 0.3469, Loss: 4.7248
Epoch   0 Batch    3/538 - Train Accuracy: 0.2643, Validation Accuracy: 0.3464, Loss: 4.1286
Epoch   0 Batch    4/538 - Train Accuracy: 0.2771, Validation Accuracy: 0.3501, Loss: 3.8791
Epoch   0 Batch    5/538 - Train Accuracy: 0.3315, Validation Accuracy: 0.3833, Loss: 3.5619
Epoch   0 Batch    6/538 - Train Accuracy: 0.3179, Validation Accuracy: 0.3635, Loss: 3.4033
Epoch   0 Batch    7/538 - Train Accuracy: 0.3232, Validation Accuracy: 0.3901, Loss: 3.3550
Epoch   0 Batch    8/538 - Train Accuracy: 0.3373, Validation Accuracy: 0.4006, Loss: 3.2101
Epoch   0 Batch    9/538 - Train Accuracy: 0.3604, Validation Accuracy: 0.4169, Loss: 3.0803
Epoch   0 Batch   10/538 - Train Accuracy: 0.3410, Validation Accuracy: 0.4233, Loss: 3.1122
Epoch   0 Batch   11/538 - Train Accuracy: 0.3529, Validation Accuracy

Epoch   0 Batch   90/538 - Train Accuracy: 0.4836, Validation Accuracy: 0.5107, Loss: 1.2929
Epoch   0 Batch   91/538 - Train Accuracy: 0.3758, Validation Accuracy: 0.4508, Loss: 1.3274
Epoch   0 Batch   92/538 - Train Accuracy: 0.4125, Validation Accuracy: 0.4627, Loss: 1.3053
Epoch   0 Batch   93/538 - Train Accuracy: 0.4572, Validation Accuracy: 0.5114, Loss: 1.2807
Epoch   0 Batch   94/538 - Train Accuracy: 0.4900, Validation Accuracy: 0.5220, Loss: 1.2833
Epoch   0 Batch   95/538 - Train Accuracy: 0.5115, Validation Accuracy: 0.5027, Loss: 1.1682
Epoch   0 Batch   96/538 - Train Accuracy: 0.4593, Validation Accuracy: 0.4860, Loss: 1.1848
Epoch   0 Batch   97/538 - Train Accuracy: 0.4475, Validation Accuracy: 0.5121, Loss: 1.2420
Epoch   0 Batch   98/538 - Train Accuracy: 0.5126, Validation Accuracy: 0.5194, Loss: 1.1637
Epoch   0 Batch   99/538 - Train Accuracy: 0.4646, Validation Accuracy: 0.5060, Loss: 1.2647
Epoch   0 Batch  100/538 - Train Accuracy: 0.4852, Validation Accuracy

Epoch   0 Batch  179/538 - Train Accuracy: 0.5768, Validation Accuracy: 0.5819, Loss: 0.8153
Epoch   0 Batch  180/538 - Train Accuracy: 0.6057, Validation Accuracy: 0.5909, Loss: 0.7849
Epoch   0 Batch  181/538 - Train Accuracy: 0.5477, Validation Accuracy: 0.5824, Loss: 0.8420
Epoch   0 Batch  182/538 - Train Accuracy: 0.5379, Validation Accuracy: 0.5779, Loss: 0.8142
Epoch   0 Batch  183/538 - Train Accuracy: 0.5800, Validation Accuracy: 0.5799, Loss: 0.7549
Epoch   0 Batch  184/538 - Train Accuracy: 0.5874, Validation Accuracy: 0.5772, Loss: 0.7599
Epoch   0 Batch  185/538 - Train Accuracy: 0.5555, Validation Accuracy: 0.5858, Loss: 0.8007
Epoch   0 Batch  186/538 - Train Accuracy: 0.5742, Validation Accuracy: 0.5822, Loss: 0.7752
Epoch   0 Batch  187/538 - Train Accuracy: 0.5923, Validation Accuracy: 0.5819, Loss: 0.7405
Epoch   0 Batch  188/538 - Train Accuracy: 0.5719, Validation Accuracy: 0.5902, Loss: 0.7799
Epoch   0 Batch  189/538 - Train Accuracy: 0.5947, Validation Accuracy

Epoch   0 Batch  268/538 - Train Accuracy: 0.6116, Validation Accuracy: 0.6309, Loss: 0.5985
Epoch   0 Batch  269/538 - Train Accuracy: 0.6041, Validation Accuracy: 0.6325, Loss: 0.6255
Epoch   0 Batch  270/538 - Train Accuracy: 0.6031, Validation Accuracy: 0.6358, Loss: 0.6308
Epoch   0 Batch  271/538 - Train Accuracy: 0.5898, Validation Accuracy: 0.6337, Loss: 0.6410
Epoch   0 Batch  272/538 - Train Accuracy: 0.5844, Validation Accuracy: 0.6465, Loss: 0.6692
Epoch   0 Batch  273/538 - Train Accuracy: 0.6240, Validation Accuracy: 0.6426, Loss: 0.6465
Epoch   0 Batch  274/538 - Train Accuracy: 0.5900, Validation Accuracy: 0.6415, Loss: 0.6717
Epoch   0 Batch  275/538 - Train Accuracy: 0.6074, Validation Accuracy: 0.6435, Loss: 0.6506
Epoch   0 Batch  276/538 - Train Accuracy: 0.6260, Validation Accuracy: 0.6422, Loss: 0.6409
Epoch   0 Batch  277/538 - Train Accuracy: 0.6047, Validation Accuracy: 0.6412, Loss: 0.6269
Epoch   0 Batch  278/538 - Train Accuracy: 0.6160, Validation Accuracy

Epoch   0 Batch  357/538 - Train Accuracy: 0.6436, Validation Accuracy: 0.6507, Loss: 0.5433
Epoch   0 Batch  358/538 - Train Accuracy: 0.6402, Validation Accuracy: 0.6506, Loss: 0.5486
Epoch   0 Batch  359/538 - Train Accuracy: 0.6639, Validation Accuracy: 0.6515, Loss: 0.5382
Epoch   0 Batch  360/538 - Train Accuracy: 0.6381, Validation Accuracy: 0.6486, Loss: 0.5592
Epoch   0 Batch  361/538 - Train Accuracy: 0.6399, Validation Accuracy: 0.6575, Loss: 0.5364
Epoch   0 Batch  362/538 - Train Accuracy: 0.6388, Validation Accuracy: 0.6552, Loss: 0.5151
Epoch   0 Batch  363/538 - Train Accuracy: 0.6568, Validation Accuracy: 0.6612, Loss: 0.5187
Epoch   0 Batch  364/538 - Train Accuracy: 0.6229, Validation Accuracy: 0.6511, Loss: 0.5630
Epoch   0 Batch  365/538 - Train Accuracy: 0.6140, Validation Accuracy: 0.6357, Loss: 0.5295
Epoch   0 Batch  366/538 - Train Accuracy: 0.6496, Validation Accuracy: 0.6474, Loss: 0.5564
Epoch   0 Batch  367/538 - Train Accuracy: 0.6391, Validation Accuracy

...

Epoch   1 Batch  177/538 - Train Accuracy: 0.8888, Validation Accuracy: 0.8786, Loss: 0.1396
Epoch   1 Batch  178/538 - Train Accuracy: 0.8765, Validation Accuracy: 0.8873, Loss: 0.1360
Epoch   1 Batch  179/538 - Train Accuracy: 0.8898, Validation Accuracy: 0.8920, Loss: 0.1352
Epoch   1 Batch  180/538 - Train Accuracy: 0.9057, Validation Accuracy: 0.8883, Loss: 0.1282
Epoch   1 Batch  181/538 - Train Accuracy: 0.9008, Validation Accuracy: 0.8794, Loss: 0.1416
Epoch   1 Batch  182/538 - Train Accuracy: 0.9127, Validation Accuracy: 0.8745, Loss: 0.1302
Epoch   1 Batch  183/538 - Train Accuracy: 0.8984, Validation Accuracy: 0.8757, Loss: 0.1203
Epoch   1 Batch  184/538 - Train Accuracy: 0.9036, Validation Accuracy: 0.8766, Loss: 0.1300
Epoch   1 Batch  185/538 - Train Accuracy: 0.9248, Validation Accuracy: 0.8867, Loss: 0.1153
Epoch   1 Batch  186/538 - Train Accuracy: 0.9094, Validation Accuracy: 0.8949, Loss: 0.1202
Epoch   1 Batch  187/538 - Train Accuracy: 0.9007, Validation Accuracy

Epoch   1 Batch  266/538 - Train Accuracy: 0.9083, Validation Accuracy: 0.9070, Loss: 0.1014
Epoch   1 Batch  267/538 - Train Accuracy: 0.9145, Validation Accuracy: 0.9112, Loss: 0.1075
Epoch   1 Batch  268/538 - Train Accuracy: 0.9401, Validation Accuracy: 0.9064, Loss: 0.0808
Epoch   1 Batch  269/538 - Train Accuracy: 0.9141, Validation Accuracy: 0.9100, Loss: 0.1064
Epoch   1 Batch  270/538 - Train Accuracy: 0.9109, Validation Accuracy: 0.9073, Loss: 0.0979
Epoch   1 Batch  271/538 - Train Accuracy: 0.9246, Validation Accuracy: 0.9169, Loss: 0.0856
Epoch   1 Batch  272/538 - Train Accuracy: 0.8891, Validation Accuracy: 0.9197, Loss: 0.1097
Epoch   1 Batch  273/538 - Train Accuracy: 0.9154, Validation Accuracy: 0.9258, Loss: 0.1060
Epoch   1 Batch  274/538 - Train Accuracy: 0.8836, Validation Accuracy: 0.9224, Loss: 0.1033
Epoch   1 Batch  275/538 - Train Accuracy: 0.9123, Validation Accuracy: 0.9096, Loss: 0.0987
Epoch   1 Batch  276/538 - Train Accuracy: 0.8957, Validation Accuracy

Epoch   1 Batch  355/538 - Train Accuracy: 0.9277, Validation Accuracy: 0.9233, Loss: 0.0835
Epoch   1 Batch  356/538 - Train Accuracy: 0.9327, Validation Accuracy: 0.9201, Loss: 0.0680
Epoch   1 Batch  357/538 - Train Accuracy: 0.9229, Validation Accuracy: 0.9148, Loss: 0.0789
Epoch   1 Batch  358/538 - Train Accuracy: 0.9225, Validation Accuracy: 0.9212, Loss: 0.0689
Epoch   1 Batch  359/538 - Train Accuracy: 0.9094, Validation Accuracy: 0.9221, Loss: 0.0820
Epoch   1 Batch  360/538 - Train Accuracy: 0.9164, Validation Accuracy: 0.9197, Loss: 0.0775
Epoch   1 Batch  361/538 - Train Accuracy: 0.9302, Validation Accuracy: 0.9201, Loss: 0.0771
Epoch   1 Batch  362/538 - Train Accuracy: 0.9364, Validation Accuracy: 0.9137, Loss: 0.0697
Epoch   1 Batch  363/538 - Train Accuracy: 0.9394, Validation Accuracy: 0.9144, Loss: 0.0778
Epoch   1 Batch  364/538 - Train Accuracy: 0.9172, Validation Accuracy: 0.9183, Loss: 0.0926
Epoch   1 Batch  365/538 - Train Accuracy: 0.9200, Validation Accuracy

Epoch   1 Batch  444/538 - Train Accuracy: 0.9299, Validation Accuracy: 0.9141, Loss: 0.0656
Epoch   1 Batch  445/538 - Train Accuracy: 0.9514, Validation Accuracy: 0.9153, Loss: 0.0558
Epoch   1 Batch  446/538 - Train Accuracy: 0.9401, Validation Accuracy: 0.9192, Loss: 0.0657
Epoch   1 Batch  447/538 - Train Accuracy: 0.9266, Validation Accuracy: 0.9165, Loss: 0.0670
Epoch   1 Batch  448/538 - Train Accuracy: 0.9275, Validation Accuracy: 0.9231, Loss: 0.0616
Epoch   1 Batch  449/538 - Train Accuracy: 0.9361, Validation Accuracy: 0.9144, Loss: 0.0770
Epoch   1 Batch  450/538 - Train Accuracy: 0.9206, Validation Accuracy: 0.9178, Loss: 0.0818
Epoch   1 Batch  451/538 - Train Accuracy: 0.9148, Validation Accuracy: 0.9260, Loss: 0.0706
Epoch   1 Batch  452/538 - Train Accuracy: 0.9434, Validation Accuracy: 0.9270, Loss: 0.0591
Epoch   1 Batch  453/538 - Train Accuracy: 0.9367, Validation Accuracy: 0.9254, Loss: 0.0773
Epoch   1 Batch  454/538 - Train Accuracy: 0.9345, Validation Accuracy

Epoch   1 Batch  533/538 - Train Accuracy: 0.9482, Validation Accuracy: 0.9405, Loss: 0.0597
Epoch   1 Batch  534/538 - Train Accuracy: 0.9455, Validation Accuracy: 0.9350, Loss: 0.0513
Epoch   1 Batch  535/538 - Train Accuracy: 0.9436, Validation Accuracy: 0.9350, Loss: 0.0529
Epoch   1 Batch  536/538 - Train Accuracy: 0.9397, Validation Accuracy: 0.9370, Loss: 0.0699
Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [32]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [33]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [34]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    
    sentence = sentence.lower()
    words = sentence.split()
    word_id_list = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in words]
    return word_id_list


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [40]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [151, 5, 69, 158, 225, 91, 222]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [225, 159, 159, 143, 286, 351, 1]
  French Words: il un un camion jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.